# Workflow for searching new recall samples

* Explain what this workflow does
* Explain what what one should do with output files 

## I. Setup

* Explain why we need subsamples and which files define them.
* Clean the cells so that there only these that are absolutely necesary
* You can add asserts to diagnose potential errors.

**TODO:** remove these lines when the file is complete. It is not a goot stile in production code.  

In [118]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Loading the corpus

**TODO:** The following cell is evil. You will commit the password sooner or later

* Introduce configparser based solution

https://github.com/estnltk/syntax_experiments/blob/syntax_consistency/collection_splitting/1_collection_splitting.py
Lines 20-43

Push this into EstNLTK so that we do not have to reinvent this every time

In [30]:
from estnltk import Text
from estnltk.storage.postgres import PostgresStorage, create_schema


storage = PostgresStorage(host='postgres.keeleressursid.ee',
                          port=5432,
                          dbname='estonian-text-corpora',
                          user='rasmusm',
                          password='',
                          schema='estonian_text_corpora',
                          role='estonian_text_corpora_read',
                          temporary=False)

display(storage)

collection = storage['koondkorpus_sentences']

collection.selected_layers = ['v171_named_entities','v172_geo_terms']

INFO:storage.py:58: connecting to host: 'postgres.keeleressursid.ee', port: 5432, dbname: 'estonian-text-corpora', user: 'rasmusm'
INFO:storage.py:108: schema: 'estonian_text_corpora', temporary: False, role: 'estonian_text_corpora_read'


**TODO:** Explain why do you do the following thing and what one should to with the resulting file. 

In [123]:
terms = []
with open('geo_terms.txt','r',encoding='UTF-8') as f:
    term = f.readline()
    while term is not '':
        terms.append(term.strip())
        term = f.readline()

### Local copy of Span Sampler

**TODO:** This header is uninformative. Say Why do you do this?
What are the steps to complete this!

In [ ]:
    def attribute_locations_creation(self):
        self.conn.commit()
        self.cur.execute("""SELECT EXISTS (
           SELECT FROM information_schema.tables 
           WHERE  table_schema = 'public'
           AND    table_name   = 'attribute_locations'
           );""")
        res = self.cur.fetchall()
        if not res[0][0]:
            self.cur.execute("CREATE TABLE attribute_locations (layer_id integer, attribute_value varchar, indices integer[], count integer);")
            self.conn.commit()
            for term in terms:
                q = LayerQuery('v172_geo_terms', lemma=term)
                for key, txt in tqdm(collection.select(query=q,layers=['v172_geo_terms'])):
                    indices = [i for i, nertag in enumerate(txt['v172_geo_terms']['lemma']) if nertag[0] ==term]
                    self.cur.execute("INSERT INTO attribute_locations (layer_id, attribute_value,indices,count) VALUES (%s, %s, %s, %s)",(key, term, indices,len(indices)))

        self.conn.commit()

**TODO:** Abstract this away into EstNLTK library. Qoes probably under storage

In [124]:
from random import sample, choices
from estnltk.storage.postgres import LayerQuery, IndexQuery
from tqdm import tqdm

class SpanSampler:
    
    def __init__(self, storage, collection, layer, attribute):
        self.storage = storage
        self.conn = storage.conn
        self.cur = self.conn.cursor()
        self.collection = collection
        self.layer = layer
        self.attribute = attribute
    
    def __call__(self, count, attribute, return_index=False, with_replacement=True): 
        # Returns iterator of type Text, Span or int, Text, span
        # count determines the number of samples
        # with replacement means that same span can be sampled several times
        self.conn.commit()
        self.create_sampling_matrix(attribute)
        indices = self.find_sampled_indices(count,with_replacement)
        result_list = []
        only_txt_index = [idx[1] for idx in indices]
        texts = list(collection.select( query=IndexQuery(only_txt_index),layers=[self.layer],return_index=True ))
        for text in texts:
            idx = [index for index in indices if text[0] == index[1]][0]
            if return_index:
                result_list.append((idx[0],text[1],text[1][self.layer][idx[2]]))
            else:
                result_list.append((text[1],text[1][self.layer][idx[2]]))
        self.clear_sampling_matrix()
        return result_list
    

        
    def create_sampling_matrix(self,attribute_val):
        self.cur.execute("CREATE TABLE sampling_matrix (id serial, layer integer, layer_index integer);")
        self.cur.execute("INSERT INTO sampling_matrix (layer,layer_index) (SELECT layer_id as layer, unnest(indices) as layer_index FROM attribute_locations WHERE attribute_value IN " + str(attribute_val) + ");")
        self.conn.commit()
    
    def find_sampled_indices(self,count,with_replacement):
        self.cur.execute("SELECT COUNT(*) FROM sampling_matrix;")
        span_count = self.cur.fetchall()[0][0]
        self.conn.commit()
        if with_replacement:
            sampled = choices(range(span_count),k=count)
        else:
            sampled = sample(range(span_count),count)
        self.cur.execute("SELECT * FROM sampling_matrix WHERE id IN " + str(tuple(sampled)) + ';')
        return self.cur.fetchall()
    
    def clear_sampling_matrix(self):
        self.conn.commit()
        self.cur.execute("DROP TABLE sampling_matrix;")
        self.conn.commit()
        

**TODO:** Why do we need a separate Postgre installation. Can't we use a different schema. Again pack the arguments into config files.
If you do that then you have to explain only what must be possible for this schema configuration. It would make a create sense to have a single configuration file with different section names.

In [1]:
# Second storage to keep the temporary lists used for sampling
localstorage = PostgresStorage(host='localhost',
                          port=5432,
                          dbname='ner_test',
                          user='postgres',
                          password='dbpass',
                          pgpass_file='~/.pgpass',
                          schema='my_schema',
                          role=None,
                          temporary=False)

NameError: name 'PostgresStorage' is not defined

**TODO:** Where do you use the this cursor? If not delete it.  

In [7]:
mycur = localstorage.conn.cursor()

**TODO:** Explain what do you do with the samper

In [126]:
sampler = SpanSampler(storage=localstorage,collection=collection, layer='v172_geo_terms',attribute='lemma')

#localstorage.conn.commit()

#sampler.clear_sampling_matrix()

samples = sampler(count=1000,attribute=tuple(filtered_terms))

display(samples[:3])

## II. Creating unlabelled samples 

* Explain what do you do with samples after they are stored.
* Explain how to check that new samples are not already sampled. 

In [156]:

import pickle

with open("1000_ulejaanud.pickle",'wb') as f:
    pickle.dump(samples,f)

In [157]:
from copy import copy

for text, sample_span in samples:
    for span in copy(text.v172_geo_terms.spans):
        if span != sample_span:
            text.v172_geo_terms.remove_span(span)

### Getting sentences to labelstudio format

**TODO:** What do with the result and combine cells into a single cell block.

In [130]:
from estnltk.converters.label_studio.label_studio import LabelStudioExporter

In [158]:
exporter = LabelStudioExporter("koond_1000_ulejaanud.json",'v172_geo_terms',checkbox=True)

In [159]:
print(exporter.labeling_interface)


        <View>
            <Labels name="label" toName="text">
	<Label value="v172_geo_terms" background="#F888F5"/> 

            </Labels>
        <Text name="text" value="$text"/>
            <Header value="Are the annotations correct?"/>
                <Choices name="review" toName="text">
                    <Choice value="yes"/>
                    <Choice value="no"/>
                </Choices>
            </View>


In [160]:
only_texts = [sample[0] for sample in samples]

In [161]:
exporter.convert(only_texts,append=False)